In [20]:
from src.model import LIFNetwork as LIF
import numpy as np

In [21]:
## Parameters (Change the following parameters to tune)
glove_stim_freq = 50 
n_finger = 10     # Number of fingers
n_neurons = 200   # Number of neurons
proba_conn = 0.2  # Probability of connection for finger-to-neuron 
glove_stim_duration = 20000
kuramato_period = 200
kuramato_lookBack = 1000

## Tracker Variables
x = []  # Tracker of variable o ---> ??? WHY?
track_post_sim_W_running_mean = np.array([])  # Tracker of running average of mean_W2
track_kuramato_order = np.array([])  # Tracker of Kuramato order 
track_kuramato_order_running_mean = np.array([]) # Tracker of running average of Kuramato order
o = 0   # Counter of the number-th run (why not just use the i from the for loop?)
track_post_sim_W_mean = np.array([])  # Holder of mean_W2
finger_stim = np.zeros([glove_stim_duration, n_finger])  # Tracker of finger stimulation over time
finger_neuron_conn = np.zeros([n_finger, n_neurons])     # Tracker of finger-Neuron connection stimulation
I = np.zeros([glove_stim_duration, n_neurons])           # Tracker of input to neuron (generated by glove stimulation)

## LIF instantiation
LIF = LIF.LIF_Network(n_neurons = n_neurons)
dt = LIF.dt  # Time step

## Stimulation variables
n_stim_per_step = int((glove_stim_duration/glove_stim_freq) / dt)    # Number of stimulation per Euler-step
stim_length_per_finger = int(glove_stim_duration/n_finger)           # Total stimulation length per finger - Assuming each finger receives equal stimulation? 

## Finger stimulation order
stim_order = np.arange(n_finger)
np.random.shuffle(stim_order)


## Generating finger to neuron stimulation, **based on probability**
##### NOTES (Tony) #####
##### Instead of using nested for-loops, directly generate a matrix for efficiency.
#####
for neuron_idx in range(n_neurons):
  for finger_idx in range(n_finger):
    if np.random.rand() < proba_conn:
      finger_neuron_conn[finger_idx, neuron_idx] = 1
## ==Faster way:==
# finger_neuron_conn = np.random.rand(n_finger, n_neurons)
# finger_neuron_conn = finger_neuron_conn - proba_conn
# finger_neuron_conn = (finger_neuron_conn <= 0)*1
# print(finger_neuron_conn)

## Update tracker to mark whether a neuron receives current from finger-stimulation
##### NOTES (Tony) ###### 
##### Correction made below, the original variable `num` and now named 
##### `n_stim_per_step` seems to have been confused leading to the `time_idx` to 
##### be jumped every `num` millisecond.
#####
# for finger_idx in range(0, n_finger):
#   for i in range(stim_length_per_finger * (finger_idx), 
#                  stim_length_per_finger * (finger_idx + 1), 
#                  n_stim_per_step): 
#     if i < glove_stim_duration:
#       finger_stim[i][stim_order[finger_idx]] = 1          # Update the finger stimulation tracker
#       I[i] [finger_neuron_conn[finger_idx,:]>0] = 1       # Based on finger_neuron_conn, marking whether a neuron receives input form the finger stimulation
## ==CORRECTION:==
for finger_idx in range(0, n_finger):
  for time_idx in range(stim_length_per_finger * (finger_idx), 
                 stim_length_per_finger * (finger_idx + 1)): 
    if time_idx < glove_stim_duration:
      finger_stim[time_idx][stim_order[finger_idx]] = 1          # Update the finger stimulation tracker
      I[time_idx][finger_neuron_conn[finger_idx,:]>0] = 1        # Based on finger_neuron_conn, marking whether a neuron receives input form the finger stimulation


## RUNNING GLOVE STIMULATION SIMULATION ##

##### NOTES (TONY) #####
##### I am going to assume that we are trying to see the snapshots of weight 
##### change throughout the glove stimulation to see how the weight changes 
##### things. Instead, the code below is running 240 full-on glove stimulations.
##### Observing the following cells, through `T*dt` it seems that the original 
##### intent was to offset `LIF_Network.stimulate()`'s turning of sim_duration
##### to Euler timesteps.
for i in range(240):
    ## Pre simulation network connection weights
    pre_sim_W = np.copy(LIF.network_W)
    pre_sim_W_mean = pre_sim_W[pre_sim_W>0].mean()

    ##### NOTES (Tony) ##### 
    ##### The second argument of plotter2 is sim_duration, feeding directly 
    ##### into the LIF_Network.simulate() argument. The original argument name
    ##### was ambiguous and thus may have led to the misunderstanding of 
    ##### LIF_Network.stimulate()'s second argument as number of Euler steps.
    # plotter2(LIF, glove_stim_duration*dt, I, pN = 5)  # plotter2 function is just being used to simulate
    ## CORRECTION
    # plotter2(LIF, glove_stim_duration, I, pN = 5)  # plotter2 function is just being used to simulate
    ## Replace plotter2 - (plotter2 is just running simulation, thus just run simulations directly on LIF)
    LIF.simulate(sim_duration = glove_stim_duration * dt, 
                 epoch_current_input = I)

    ## Post simulation network connection weights
    post_sim_W = LIF.network_W
    post_sim_W_mean = post_sim_W[post_sim_W > 0].mean()

    ## Network connection weight difference (post - pre)
    mean_W_diff = post_sim_W_mean - pre_sim_W_mean

    ## Calculate post simulation network connect (W) mean
    track_post_sim_W_mean = np.append(track_post_sim_W_mean, post_sim_W_mean)
    
    ## Calculate post simulation kuramato-order 
    kuramato_order = LIF.kuramato(period = kuramato_period, 
                                  lookBack = kuramato_lookBack)
    track_kuramato_order = np.append(track_kuramato_order, kuramato_order)

    ## Calculate post simulation W running mean
    running_mean = (sum(track_post_sim_W_mean)
                    / len(track_post_sim_W_mean))
    track_post_sim_W_running_mean = np.append(track_post_sim_W_running_mean, 
                                              running_mean)
    
    ## Calculate post simulation kuramato-order running mean
    running_kuramato = (sum(track_kuramato_order) 
                        / len(track_kuramato_order))
    track_kuramato_order_running_mean = np.append(track_kuramato_order_running_mean, 
                                                  running_kuramato)
    
    x.append(o)
    print(o)
    fig, ax = plt.subplots()
    ax.plot(x, track_post_sim_W_running_mean, color='green', label='Running W-mean')
    ax.plot(x, track_kuramato_order_running_mean, color='red', label='Kuramato Running mean')
    ax.legend()
    fig.show()
    # plt.plot(x,y, color="green")  # running average of mean_W2 vs euler-step
    # plt.plot(x,z, color="red")  # running average of Kuramato mean vs euler-step
    # plt.show()
    o=o+1  # increment the euler-step

KeyboardInterrupt: 